In [12]:
# import libraries
import sys
import os
import h5py
import pandas as pd
import matplotlib.pylab as plt
from jupyterlab_h5web import H5Web
# import modules with the functionalities offered by CompositionSpace
from compositionspace.utils import get_file_size
from compositionspace.io import get_reconstructed_positions, get_iontypes, get_ranging_info
from compositionspace.preparation import ProcessPreparation
from compositionspace.segmentation import ProcessSegmentation
from compositionspace.clustering import ProcessClustering

In [13]:
# ! pip list  
MY_PROCESSED_DATA_PATH = f"{os.getcwd()}"
print(f"Executing compositionspace in the following working directory: {os.getcwd()}")

Executing compositionspace in the following working directory: /home/kaiobach/Research/hu_hu_hu/sprint22/conda-compspace-step01/CompositionSpace


## Load reconstruction and ranging

In [19]:
workdir = "/home/kaiobach/Research/paraprobe-toolbox/teaching/example_analyses/iuc09_saksena"
# workdir = "/home/kaiobach/Research/paraprobe-toolbox/teaching/example_analyses/usa_denton_smith"
simid = 2
RECONSTRUCTION_AND_RANGING = (f"{workdir}/..")
RECONSTRUCTION_AND_RANGING = (f"{workdir}/PARAPROBE.Transcoder.Results.SimID.1.nxs",
                              f"{workdir}/PARAPROBE.Ranger.Results.SimID.1.nxs")
config_file_path = f"{MY_PROCESSED_DATA_PATH}/experiment_params.yaml"
results_file_path = f"{MY_PROCESSED_DATA_PATH}/CompositionSpace.Results.{simid}.nxs"

In [20]:
# H5Web(RECONSTRUCTION_AND_RANGING[0])
# H5Web(RECONSTRUCTION_AND_RANGING[1])

In [21]:
(xyz_val, xyz_unit) = get_reconstructed_positions(RECONSTRUCTION_AND_RANGING[0])
ityp_info, nochrg_ityp_info, elements = get_ranging_info(RECONSTRUCTION_AND_RANGING[0], verbose=True)
(ityp_val, ityp_unit) = get_iontypes(RECONSTRUCTION_AND_RANGING[1])

Load reconstructed positions shape (69507210, 3), type <class 'numpy.ndarray'>, dtype float32
111 iontypes distinguished:
	ion0, ('unknown iontype', 0)
	ion1, ('Al ++', 1)
	ion2, ('Al +', 2)
	ion3, ('Al +++', 3)
	ion4, ('Si ++', 4)
	ion5, ('Si ++', 5)
	ion6, ('Si ++', 6)
	ion7, ('Ti ++', 7)
	ion8, ('Ti ++', 8)
	ion9, ('Ti ++', 9)
	ion10, ('Ti ++', 10)
	ion11, ('Cr ++', 11)
	ion12, ('Cr ++', 12)
	ion13, ('Cr ++', 13)
	ion14, ('Cr +', 14)
	ion15, ('Cr +', 15)
	ion16, ('Cr +', 16)
	ion17, ('Ni ++', 17)
	ion18, ('Ni ++', 18)
	ion19, ('Ni +', 19)
	ion20, ('Ni +', 20)
	ion21, ('Ni +', 21)
	ion22, ('Ni ++', 22)
	ion23, ('Ni ++', 23)
	ion24, ('Ni ++', 24)
	ion25, ('Ni +', 25)
	ion26, ('Ni +', 26)
	ion27, ('Co ++', 27)
	ion28, ('Co +', 28)
	ion29, ('Ta +++', 29)
	ion30, ('Ta ++', 30)
	ion31, ('W +++', 31)
	ion32, ('W +++', 32)
	ion33, ('W ++', 33)
	ion34, ('W ++', 34)
	ion35, ('W ++', 35)
	ion36, ('W ++', 36)
	ion37, ('W +', 37)
	ion38, ('W +', 38)
	ion39, ('W +', 39)
	ion40, ('W +', 40)
	ion41

## Voxelize with rectangular transfer function without creating slices

In [22]:
voxelize = ProcessPreparation(config_file_path, results_file_path, verbose=True)
voxelize.init_ranging(ityp_info, elements)
voxelize.write_init_results()
voxelize.define_voxelization_grid(xyz_val)
voxelize.define_lookup_table(ityp_val)
voxelize.write_voxelization_grid_info()
voxelize.write_voxelization_results()

[[ 3.4028235e+38 -3.4028235e+38]
 [ 3.4028235e+38 -3.4028235e+38]
 [ 3.4028235e+38 -3.4028235e+38]]
shape (69507210,)
dim 0
	np.min(xyz[:, axis_id]) -54.959815979003906 >>>> -57.0
	np.max(xyz[:, axis_id]) 52.8397102355957 >>>> 55.0
	self.aabb3d axis_id 0, [-57.  55.], extent 112
	[-56. -55. -54. -53. -52. -51. -50. -49. -48. -47. -46. -45. -44. -43.
 -42. -41. -40. -39. -38. -37. -36. -35. -34. -33. -32. -31. -30. -29.
 -28. -27. -26. -25. -24. -23. -22. -21. -20. -19. -18. -17. -16. -15.
 -14. -13. -12. -11. -10.  -9.  -8.  -7.  -6.  -5.  -4.  -3.  -2.  -1.
   0.   1.   2.   3.   4.   5.   6.   7.   8.   9.  10.  11.  12.  13.
  14.  15.  16.  17.  18.  19.  20.  21.  22.  23.  24.  25.  26.  27.
  28.  29.  30.  31.  32.  33.  34.  35.  36.  37.  38.  39.  40.  41.
  42.  43.  44.  45.  46.  47.  48.  49.  50.  51.  52.  53.  54.  55.]
dim 1
	np.min(xyz[:, axis_id]) -58.34532165527344 >>>> -61.0
	np.max(xyz[:, axis_id]) 53.82450485229492 >>>> 56.0
	self.aabb3d axis_id 1, [-61.  56.],

In [23]:
get_file_size(results_file_path)
# H5Web(results_file_path)

211.929 MiB


Voxelization based on element types not on iontypes.

## Segmentation PCA and IC minimization

In [24]:
segmentation = ProcessSegmentation(
    config_file_path,
    results_file_path,
    entry_id=1,
    verbose=True)
segmentation.perform_pca_and_write_results()
segmentation.perform_bics_minimization_and_write_results()

Composition matrix has 22 chemical classes
Composition matrix has 22 chemical classes
GaussianMixture ML analysis with n_cluster 1
GaussianMixture ML analysis with n_cluster 2
GaussianMixture ML analysis with n_cluster 3
GaussianMixture ML analysis with n_cluster 4
GaussianMixture ML analysis with n_cluster 5


In [25]:
get_file_size(results_file_path)
# H5Web(results_file_path)

213.26 MiB


<div class="alert alert-block alert-danger">
Discussion points:<br>
* Modify NXapm_composition_space<br>
* What to show how to show?<br>
* Number of cluster vs Number of clusters? wording...?<br>
* Why to run the gm several times, ones in the bics loop ones in get composition cluster files 
</div>

## DBScan clustering

In [26]:
clustering = ProcessClustering(
    config_file_path,
    results_file_path,
    entry_id=1,
    verbose=True)
clustering.run_and_write_results()

DBScan configuration: eps 3 nm, min_samples 5
['cluster_analysis0', 'cluster_analysis1', 'cluster_analysis2', 'cluster_analysis3', 'cluster_analysis4', 'sequence_index', 'summary']
cluster_analysis0
ic_run_id 0 >>>>
np.shape(all_vxl_pos) (2725632, 3) list(set(phase_identifier) [0]
	Loop 0
	np.shape(trg_vxl_pos) (2725632, 3)
	np.shape(trg_vxl_idx) (2725632,)
	1
	type(db.labels_) <class 'numpy.ndarray'> dtype int64
[0]
cluster_analysis1
ic_run_id 1 >>>>
np.shape(all_vxl_pos) (2725632, 3) list(set(phase_identifier) [0, 1]
	Loop 0
	np.shape(trg_vxl_pos) (1379713, 3)
	np.shape(trg_vxl_idx) (1379713,)
	1
	type(db.labels_) <class 'numpy.ndarray'> dtype int64
[0]
	Loop 1
	np.shape(trg_vxl_pos) (1345919, 3)
	np.shape(trg_vxl_idx) (1345919,)
	1
	type(db.labels_) <class 'numpy.ndarray'> dtype int64
[0]
cluster_analysis2
ic_run_id 2 >>>>
np.shape(all_vxl_pos) (2725632, 3) list(set(phase_identifier) [0, 1, 2]
	Loop 0
	np.shape(trg_vxl_pos) (1, 3)
	np.shape(trg_vxl_idx) (1,)
	1
	type(db.labels_) <cl

In [27]:
get_file_size(results_file_path)
H5Web(results_file_path)

233.976 MiB


<jupyterlab_h5web.widget.H5Web object>

<div class="alert alert-block alert-danger">
Discussion points:<br>
- Tests are too specific, hardcoded file names<br>
- Readthedocs documentation needs to be updated<br>
- GM and ML models are variables collect over<br>
- Loading file formats from the community should use ifes-apt-tc-data-modeling library currently using paraprobe result<br>
- Ion handling should use ifes-apt-tc-data-modeling is not added as a dependencies and loading properly<br>
-  tests/experiment_params.json should be removed?<br>
- NeXus renaming<br>
- CompositionSpace by design does not distinguish charge states iontypes should be atomic decomposed<br>
- Why is the center of the voxel defined by the median position of the ions but not by the barycenter of the voxel (currently using voxel barycenter)<br>
- Ran 2, and even 0.5 discretization speed is comparable<br>
- Triple loop in preparation step should be replaced with more fancy numpy indexing code that I know is somewhere but I couldnt find quickly<br>
- Move test data out of this repository<br>
</div>

## Meshing

Test for now with the SiGe dataset.

In [ ]:
sige_file_path = "Output_DBSCAN_segmentation_phase_1.h5"
H5Web(sige_file_path)

In [ ]:
import h5py
import numpy as np
with h5py.File(sige_file_path, "r") as h5r:
    n_vxls = 0
    aabb3d = np.zeros((3, 2), np.float64)
    for dim in [0, 1, 2]:
        aabb3d[dim, :] = [np.finfo(np.float64).max, np.finfo(np.float64).min]
    # print(aabb3d)
    for key in h5r["1"].keys():
        for dim in [0, 1, 2]:
            mimx = (np.min(h5r["1"][key][:, dim]), np.max(h5r["1"][key][:, dim]))
            if mimx[0] <= aabb3d[dim, 0]:
                aabb3d[dim, 0] = mimx[0]
            if mimx[1] >= aabb3d[dim, 1]:
                aabb3d[dim, 1] = mimx[1]
            n_vxls += int(np.shape(h5r["1"][key])[0])
    print(aabb3d)
    print(n_vxls)
    # assume cubic vxl 2nm edge length

Assume that the data were discretized on the following rectangular grid with 2nm cubic voxel

In [ ]:
nx = int((88--96)/2)
ny = int((94--96)/2)
nz = int((0--222)/2)
grid = np.zeros((nx, ny, nz), np.uint32)
# that grid should intentionally be a cuboid to enable checking correct dimensions

def i_to_xyz(i):
    z = int(i / (nx * ny))
    rem = i - (nx * ny * z)
    y = int(rem / nx)
    x = rem - (y * nx)
    return (x, y, z)

with h5py.File(sige_file_path, "r") as h5r:
    for key in h5r["1"].keys():
        jds = np.asarray(h5r["1"][key][:, 3], np.uint32)
        for j in jds:
            x, y, z = i_to_xyz(j)
            grid[x, y, z] = int(key) + 1
        print(key)
print(np.shape(grid))
print(np.unique(grid))

In [ ]:
import h5py
with h5py.File("input.grid.nxs", "w") as h5w:
    h5w.create_dataset("/grid", compression="gzip", compression_opts=1, data=grid)

In [ ]:
H5Web("input.grid.nxs")